<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-12 15:19:46
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.26 C
-------------------
Today PnL: 4.70 L (3.86%)
Current PnL: -19.25 L (-13.65%)
CY Booked + Current PnL: -16.38 L (-11.62%)
-------------------
Total profit:  3.88 L
Total loss:  -23.13 L
-------------------
Total Booked + Current PnL: 10.60 L (9.29%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.27%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 81.13 L (64.18%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.43 C (112.97%)
Deployed:  1.14 C
Current:  1.26 C
CAGR/XIRR %: 7.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,13.86,X-MC,88.30,139845.0,25067.0,10656.0,1.58,21.84,7.62,31.13,215.0,2.35,1.14,36.22,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-0.88,H-LC,82.98,147235.0,21530.0,14458.0,3.12,17.13,9.82,28.63,116.0,1.49,1.20,31.17,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-16.57,X-LC,50.00,92487.0,17365.0,14789.0,3.45,23.12,15.99,42.81,87.0,1.17,0.76,15.99,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-13.45,M-LC,60.64,212398.0,10918.0,16291.0,2.86,5.42,7.67,13.51,91.0,0.67,1.74,16.36,XY25,NTT,BANKS
25,DMART,3484.00,5112.00,-21.57,X-LC,9.57,96602.0,12986.0,26083.0,1.33,15.53,27.00,46.73,123.0,0.50,0.79,18.25,AR,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CERA,8421.60,10420.45,-16.47,X-SC,95.74,86210.0,-23271.0,49260.0,13.71,-21.26,57.14,23.73,122.0,-0.47,0.70,30.48,X40N,ATH,CERAMICS
88,VALIANTORG,512.64,838.00,-294.01,H-SC,1.06,82446.0,-50840.0,135434.0,9.99,-38.14,164.27,63.47,83.0,-0.38,0.67,38.20,XR,NTT,CHEMICALS
16,BSOFT,628.85,844.81,-21.77,M-SC,87.23,73649.0,-36400.0,74194.0,9.47,-33.08,100.74,34.34,20.0,-0.49,0.60,21.25,XR,ATH,IT
65,SAIL,130.64,228.00,78.75,M-MC,57.45,203609.0,-21353.0,189010.0,8.47,-9.49,92.83,74.53,217.0,-0.11,1.66,18.22,XY24,BTT,STEEL
82,TITAGARH,1097.23,1548.00,-25.94,H-SC,7.45,130288.0,-61727.0,140607.0,8.33,-32.15,107.92,41.08,76.0,-0.44,1.06,10.27,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1354.50,1800.0,-39.16,H-MC,71.28,44905.0,-32302.0,57694.0,-3.71,-41.84,128.48,32.89,41.0,-0.56,0.37,23.66,XR,NTT,BANKS
67,SATIN,199.77,274.0,-17.21,H-SC,76.60,163572.0,-32402.0,105226.0,-0.87,-16.53,64.33,37.16,142.0,-0.31,1.34,22.15,XY24,NTT,FINANCE
12,BANKINDIA,113.49,190.0,-36.74,M-MC,25.53,125260.0,-3892.0,90964.0,-0.14,-3.01,72.62,67.42,71.0,-0.04,1.02,21.83,XR,NTT,BANKS
55,PFIZER,4236.33,6318.3,-15.37,H-SC,39.36,131741.0,415.0,64132.0,-0.00,0.32,48.68,49.15,102.0,0.01,1.08,8.58,X40,ATH,PHARMA
14,BERGEPAINT,558.54,680.0,-20.27,H-MC,47.87,171243.0,-5814.0,44318.0,0.67,-3.28,25.88,21.75,139.0,-0.13,1.40,22.49,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TATAMOTORS,776.02,1065.00,-52.03,M-LC,93.62,191118.0,-14527.0,91106.0,1.79,-7.06,47.67,37.24,1.0,-0.16,1.56,24.40,XY24,NTT,AUTO
2,ABB,5319.37,8953.12,-38.90,H-LC,73.40,139812.0,6828.0,84013.0,2.79,5.13,60.09,68.31,2.0,0.08,1.14,14.01,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-26.04,H-LC,48.94,122508.0,-495.0,63202.0,7.74,-0.40,51.59,50.98,3.0,-0.01,1.00,11.93,AR,ATH,REALTY
10,AWL,342.88,485.00,-69.16,H-MC,6.38,166910.0,-50819.0,141072.0,4.06,-23.34,84.52,41.45,5.0,-0.36,1.36,11.50,XY24,NTT,FMCG
92,WHIRLPOOL,1151.42,2270.00,-35.92,M-SC,92.55,161430.0,15200.0,126868.0,3.94,10.39,78.59,97.15,7.0,0.12,1.32,37.91,X40,NTT,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,13.86,X-MC,88.30,139845.0,25067.0,10656.0,1.58,21.84,7.62,31.13,215.0,2.35,1.14,36.22,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,5.95,X-MC,13.83,186793.0,31358.0,39077.0,5.00,20.17,20.92,45.32,198.0,0.80,1.53,27.10,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-16.57,X-LC,50.00,92487.0,17365.0,14789.0,3.45,23.12,15.99,42.81,87.0,1.17,0.76,15.99,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-3.31,M-LC,85.11,186786.0,35580.0,30091.0,4.37,23.53,16.11,43.43,37.0,1.18,1.53,29.47,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,POLYCAB,5005.92,7541.35,7.21,H-MC,89.36,167594.0,27428.0,43558.0,3.80,19.57,25.99,50.65,54.0,0.63,1.37,27.91,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,-0.62,H-SC,72.34,161590.0,10226.0,35550.0,6.86,6.76,22.00,30.25,131.0,0.29,1.32,27.89,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-3.31,M-LC,85.11,186786.0,35580.0,30091.0,4.37,23.53,16.11,43.43,37.0,1.18,1.53,29.47,XY24,NTT,MISC
93,WIPRO,243.46,420.00,-6.99,M-LC,62.77,159731.0,8786.0,100662.0,6.45,5.82,63.02,72.51,58.0,0.09,1.31,20.26,XR,NTT,IT
18,CAMS,3643.00,5250.99,-8.44,H-SC,11.70,102830.0,826.0,44196.0,5.54,0.81,42.98,44.14,47.0,0.02,0.84,19.12,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-6.99,M-LC,62.77,159731.0,8786.0,100662.0,6.45,5.82,63.02,72.51,58.0,0.09,1.31,20.26,XR,NTT,IT
18,CAMS,3643.00,5250.99,-8.44,H-SC,11.70,102830.0,826.0,44196.0,5.54,0.81,42.98,44.14,47.0,0.02,0.84,19.12,XR,ATH,MISC
37,INDIAMART,2311.97,4911.36,-53.33,H-SC,75.53,101024.0,-703.0,115076.0,1.79,-0.69,113.91,112.43,NaN,-0.01,0.83,19.23,AR,ATH,MISC
91,VOLTAS,1272.00,1929.20,-30.18,H-MC,22.34,139447.0,-473.0,72763.0,2.60,-0.34,52.18,51.67,19.0,-0.01,1.14,5.33,AR,ATH,AC
12,BANKINDIA,113.49,190.00,-36.74,M-MC,25.53,125260.0,-3892.0,90964.0,-0.14,-3.01,72.62,67.42,71.0,-0.04,1.02,21.83,XR,NTT,BANKS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,52.77,60.00,34.19,M-SC,77.66,115965.0,-26514.0,46038.0,5.45,-18.61,39.70,13.70,219.0,-0.58,0.95,36.87,X40N,NTT,BANKS
53,MRF,114777.60,150504.14,13.86,X-MC,88.30,139845.0,25067.0,10656.0,1.58,21.84,7.62,31.13,215.0,2.35,1.14,36.22,AR,ATH,TYRES
86,UNIONBANK,123.87,163.00,-17.18,M-LC,59.57,145741.0,4901.0,39583.0,4.30,3.48,27.16,31.59,83.0,0.12,1.19,26.60,XY24,NTT,BANKS
69,SBILIFE,1496.49,1924.99,-0.88,H-LC,82.98,147235.0,21530.0,14458.0,3.12,17.13,9.82,28.63,116.0,1.49,1.20,31.17,AR,ATH,INSURANCE
52,MEDANTA,1022.60,1486.00,5.95,X-MC,13.83,186793.0,31358.0,39077.0,5.00,20.17,20.92,45.32,198.0,0.80,1.53,27.10,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.00,-805.27,H-SC,100.00,67187.0,-28129.0,72824.0,4.27,-29.51,108.39,46.89,51.0,-0.39,0.55,32.52,X40N,NTT,MISC
79,TATAELXSI,7332.28,9000.00,-18.50,X-MC,98.94,85568.0,-17084.0,40431.0,6.52,-16.64,47.25,22.74,33.0,-0.42,0.70,28.62,X40N,NTT,IT
58,QUESS,711.66,986.00,NaN,NaN,97.87,78063.0,-77791.0,137875.0,8.28,-49.91,176.62,38.55,NaN,-0.56,0.64,19.53,XY24,NTT,MISC
1,AARTIIND,487.04,919.00,36.48,M-SC,96.81,78306.0,-3030.0,75166.0,3.83,-3.72,95.99,88.69,118.0,-0.04,0.64,32.05,XR,NTT,CHEMICALS
19,CERA,8421.60,10420.45,-16.47,X-SC,95.74,86210.0,-23271.0,49260.0,13.71,-21.26,57.14,23.73,122.0,-0.47,0.70,30.48,X40N,ATH,CERAMICS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.57
1,25,41.35
2,50,69.63


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.62
X40     14.64
X40N    14.01
AR      13.79
XY25    10.35
XR      10.26
X200     1.21
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.01
X-LC    18.57
M-SC    16.47
M-LC     8.48
H-MC     7.87
X-MC     6.82
H-LC     5.67
M-MC     5.50
X-SC     2.81
L-SC     2.24
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.93,-1.64,33.97
BANKS,11.06,-11.26,65.83
IT,9.35,-20.97,88.45
FINANCE,7.31,-23.78,79.64
MISC,7.16,-29.30,84.93
INSURANCE,6.03,-4.60,36.14
PAINTS,5.62,-19.58,54.97
ELECTRICAL,4.09,-10.86,67.35
REFINERIES,3.71,10.38,6.14


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3113735.0,27
XR,1287062.0,15
AR,1267063.0,15
X40N,1218606.0,17
X40,804336.0,12
SR,209061.0,2
XY25,143585.0,5
X200,69437.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2494916.0,22
M-SC,1989743.0,20
X-LC,710625.0,12
M-MC,593446.0,5
H-MC,433683.0,7
X-SC,364390.0,4
M-LC,343981.0,6
L-SC,322019.0,4
X-MC,320998.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      949435.0      6
M-SC       XY24      795508.0      6
H-SC       X40N      568985.0      7
           AR        559663.0      3
M-SC       XR        436363.0      5
M-MC       XY24      417455.0      3
M-SC       X40N      378781.0      5
X-LC       X40       365186.0      5
H-SC       XR        311850.0      4
M-LC       XY24      227028.0      4
M-SC       X40       200032.0      2
H-LC       AR        193343.0      4
X-LC       AR        186199.0      3
H-MC       XY24      185390.0      2
L-SC       XR        164689.0      2
X-MC       XY24      140171.0      2
X-SC       XR        124840.0      1
           SR        122582.0      1
           X40N      116968.0      2
X-MC       X40N      110314.0      2
H-SC       X40       104983.0      2
M-LC       XR        100662.0      1
M-SC       AR         92580.0      1
M-MC       XR         90964.0      1
L-SC       XY24       90498.0      1
X-LC       XY25       89803.0      3
M-SC       SR         86479.0      1
M-MC       AR         85027.0      1
H-MC       X40        74278.0      2
           AR         72763.0      1
X-LC       X200       69437.0      1
L-SC       AR         66832.0      1
X-MC       X40        59857.0      1
H-MC       XR         57694.0      1
           X40N       43558.0      1
H-LC       XY25       37491.0      1
M-LC       XY25       16291.0      1
X-MC       AR         10656.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
